<a href="https://colab.research.google.com/github/vantainguyen/A-B-Testing/blob/main/V_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
# Parameters for the two model we want to build
OUTPUT_CHANNELS = 3

learning_rate1=0.001
learning_rate2=0.01

optimizer1 = tf.keras.optimizers.Adam(learning_rate=learning_rate1)
optimizer2 = tf.keras.optimizers.Adam(learning_rate=learning_rate2)

batch_size1 = 8
batch_size2 = 8

kernel_size1 = 3
kernel_size2 = 3

filter_base1 = 16
filter_base2 = 32

#loss1 = 'mean_squared_error'
#loss2 = 'mean_squared_error'

self_defined_loss = False # To get the root mean square error

if self_defined_loss:

  def loss_func(y_true, y_pred): 
      
      # Root mean square loss
      squared_difference = tf.square(y_true - y_pred)
      mean_square_loss = tf.reduce_mean(squared_difference, axis=-1)
      root_mean_square_loss = tf.sqrt(mean_square_loss + 1e-20)
      
      return root_mean_square_loss

else:

  loss_func = 'mean_squared_error'



epochs_train = 300

save_period = epochs_train

initializer = tf.random_normal_initializer(0., 0.02)

class V_Net(tf.keras.Model):

  self.convD1 = tf.keras.layers.Conv3D(filter_base2, kernel_size=kernel_size1, strides=(2,2,2), padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  self.convD2 = tf.keras.layers.Conv3D(filter_base2*2, kernel_size=kernel_size1, strides=(2,2,2), padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  self.convD3 = tf.keras.layers.Conv3D(filter_base2*4, kernel_size=kernel_size1, strides=(2,2,2), padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  self.convD4 = tf.keras.layers.Conv3D(filter_base2*8, kernel_size=kernel_size1, strides=(2,2,2), padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  self.convD5 = tf.keras.layers.Conv3D(filter_base2*16, kernel_size=kernel_size1, strides=(2,2,2), padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  self.convD6 = tf.keras.layers.Conv3D(filter_base2*32, kernel_size=kernel_size1, strides=(2,2,2), padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  
  self.conv_batch_norm = tf.keras.layers.BatchNormalization()
  self.conv_activation = tf.keras.layers.LeakyReLU()


  self.convU1 = tf.keras.layers.Conv3DTranspose(filter_base2*32, kernel_size=kernel_size1, strides=2, padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  self.convU2 = tf.keras.layers.Conv3DTranspose(filter_base2*16, kernel_size=kernel_size1, strides=2, padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  self.convU3 = tf.keras.layers.Conv3DTranspose(filter_base2*8, kernel_size=kernel_size1, strides=2, padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  self.convU4 = tf.keras.layers.Conv3DTranspose(filter_base2*4, kernel_size=kernel_size1, strides=2, padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  self.convU5 = tf.keras.layers.Conv3DTranspose(filter_base2*2, kernel_size=kernel_size1, strides=2, padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  
  self.concat = tf.keras.layers.Concatenate()

  self.last = tf.keras.layers.Conv3DTranspose(OUTPUT_CHANNELS, kernel_size=kernel_size1, strides=2, padding='same', 
                                      kernel_initializer=initializer, use_bias=False)
  
  # self.convU1_batch_norm = tf.keras.layers.BatchNormalization()
  # self.convU1_activation = tf.keras.layers.LeakyReLU()


  def call(self, inputs):

    x1 = self.convD1(inputs)
    x1 = self.conv_batch_norm(x1)
    x1 = self.conv_activation(x1)

    x2 = self.convD2(x1)
    x2 = self.conv_batch_norm(x2)
    x2 = self.conv_activation(x2)

    x3 = self.convD3(x2)
    x3 = self.conv_batch_norm(x3)
    x3 = self.conv_activation(x3)

    x4 = self.convD4(x3)
    x4 = self.conv_batch_norm(x4)
    x4 = self.conv_activation(x4)

    x5 = self.convD5(x4)
    x5 = self.conv_batch_norm(x5)
    x5 = self.conv_activation(x5)

    x6 = self.convD6(x5)
    x6 = self.conv_batch_norm(x6)
    x6 = self.conv_activation(x6)

    x7 = self.convU1(x6)
    x7 = self.conv_batch_norm(x7)
    x7 = self.conv_batch_norm(x7)

    
    x8 = self.concat([x7, x6])



NameError: ignored

In [1]:
from tensorflow.python.ops import array_ops
import tensorflow as tf

def image3D_gradients(image):

    """Returns image gradients (dy, dx, dz) for each color channel.
    Both output tensors have the same shape as the input: [batch_size, h, w, d
    c]. The gradient values are organized so that [I(x+1, y, z) - I(x, y, z)] is in
    location (x, y, z). That means that dz will always have zeros in the last depth, dy will always have zeros in the last row,
    and dx will always have zeros in the last column.
    Usage Example:
      ```python
      BATCH_SIZE = 1
      IMAGE_HEIGHT = 5
      IMAGE_WIDTH = 5
      IMAGE_DEPTH = 5
      CHANNELS = 1
      image = tf.reshape(tf.range(IMAGE_HEIGHT * IMAGE_WIDTH * IMAGE_DEPTH * CHANNELS,
        delta=1, dtype=tf.float32),
        shape=(BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, CHANNELS))
      dy, dx, dz = image3D_gradients(image)
      ```
    Args:
      image: Tensor with shape [batch_size, h, w, d, c].
    Returns:
      Pair of tensors (dy, dx, dz) holding the vertical, horizontal and depth image
      gradients (1-step finite difference).
    Raises:
      ValueError: If `image` is not a 5D tensor.
    """
    if image.get_shape().ndims != 5:
      raise ValueError('image3D_gradients expects a 5D tensor '
                      '[batch_size, h, w, d, c], not {}.'.format(image.get_shape()))
      
    image_shape = array_ops.shape(image)
    batch_size, height, width, depth, classes = array_ops.unstack(image_shape)

    dy = image[:, 1:, :, :, :] - image[:, :-1, :, :, :]
    dx = image[:, :, 1:, :, :] - image[:, :, :-1, :, :]
    dz = image[:, :, :, 1:, :] - image[:, :, :, :-1, :]

    # Return tensors with same size as original image by concatenating
    # zeros. Place the gradient [I(x+1,y, z) - I(x,y, z)] on the base pixel (x, y, z).
    shape = array_ops.stack([batch_size, 1, width, depth, classes])
    dy = array_ops.concat([dy, array_ops.zeros(shape, image.dtype)], 1)
    dy = array_ops.reshape(dy, image_shape)

    shape = array_ops.stack([batch_size, height, 1, depth, classes])
    dx = array_ops.concat([dx, array_ops.zeros(shape, image.dtype)], 2)
    dx = array_ops.reshape(dx, image_shape)

    shape = array_ops.stack([batch_size, height, width, 1, classes])
    dz = array_ops.concat([dz, array_ops.zeros(shape, image.dtype)], 3)
    dz = array_ops.reshape(dz, image_shape)

    return dy, dx, dz

In [12]:
image.get_shape().ndims

5

In [3]:
BATCH_SIZE = 1
IMAGE_HEIGHT = 5
IMAGE_WIDTH = 5
IMAGE_DEPTH = 5
CHANNELS = 1
image = tf.reshape(tf.range(IMAGE_HEIGHT * IMAGE_WIDTH * IMAGE_DEPTH * CHANNELS,
  delta=1, dtype=tf.float32),
  shape=(BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, CHANNELS))
dy, dx, dz = image3D_gradients(image)
print(image[0, :,:,0])

tf.Tensor(
[[[  0.]
  [  5.]
  [ 10.]
  [ 15.]
  [ 20.]]

 [[ 25.]
  [ 30.]
  [ 35.]
  [ 40.]
  [ 45.]]

 [[ 50.]
  [ 55.]
  [ 60.]
  [ 65.]
  [ 70.]]

 [[ 75.]
  [ 80.]
  [ 85.]
  [ 90.]
  [ 95.]]

 [[100.]
  [105.]
  [110.]
  [115.]
  [120.]]], shape=(5, 5, 1), dtype=float32)


In [6]:
print(dz[0, :,:,0])

tf.Tensor(
[[[1.]
  [1.]
  [1.]
  [1.]
  [1.]]

 [[1.]
  [1.]
  [1.]
  [1.]
  [1.]]

 [[1.]
  [1.]
  [1.]
  [1.]
  [1.]]

 [[1.]
  [1.]
  [1.]
  [1.]
  [1.]]

 [[1.]
  [1.]
  [1.]
  [1.]
  [1.]]], shape=(5, 5, 1), dtype=float32)
